# Problems
In Topfarm the problem is the collection/container of components and drivers. Optimizing a problem is executing a workflow - finding the best feasible solution under a series of constraints using a specific driver, and initial conditions. To the problem is also attached a plotting routine so one can follow the optimization path as well as a recorder so that intermediate variable values are accessible after the optimization is finished. The Topfarm Problem inherits its fundamental nature from the OpenMDAO problem, and is being adapted so it can connect the given workflow with the driver. For example if the user specifies a boundary constraint and this is not supported by the driver, the Problem is equipped with a penalty component that will deter the driver giving unfeasible solutions. Or if your workflow does not have gradients for all components and a gradient based driver is specified, finite differencing is applied to obtain the gradients. 

**Jump to example in this notebook**:

* [Turbine location optimization](#Turbine-location-optimization)

Make sure you first run the code below in order to set up and initialize needed variables.

**First we install topfarm and import supporting libraries in Python numpy and matplotlib**

In [1]:
%%capture
try:  # install Topfarm if needed
    import topfarm
except ModuleNotFoundError:
    !pip install topfarm
import numpy as np

In [3]:
# non-updating, inline plots
#%matplotlib inline
# ...or updating plots in new window
%matplotlib qt

In [4]:
import matplotlib.pyplot as plt 

**Next we import and initialize several functions and classes from Topfarm to set up the problem including:**
* **TopFarmProblem - overall topfarm problem class to which the objectives, design variables, and constraints are added**
* **EasyScipyOptimizeDriver - a subclass of ScipyOptimizeDriver which is configured for the given workflow**
* **get_iea37_initial, get_iea37_constraints, get_iea37_cost - functions to get the initial layout, the constraints and the cost function for the IEA task 37 benchmark example**
* **NoPlot, XYPlotComp - plotting components to visualize the results**

In [5]:
from topfarm import TopFarmProblem
from topfarm.easy_drivers import EasyScipyOptimizeDriver
from topfarm.examples.iea37 import get_iea37_initial, get_iea37_constraints, get_iea37_cost
from topfarm.plotting import NoPlot, XYPlotComp

## Turbine location optimization

This example optimizes the locations of the 9-turbine benchmark wind farm from IEA Task 37 using the provided initial locations and the `EasyScipyOptimizeDriver`. Details on the benchmark can be found in the following reference:  

* Baker et al. (2019) "Best Practices for Wake Model and Optimization Algorithm Selection in Wind Farm Layout Optimization". AIAA 2019.

In [12]:
from topfarm.examples.iea37 import *
def get_iea37_cost(n_wt=9, n_wd=16):
    """Cost component that wraps the IEA 37 AEP calculator"""
    wd = np.linspace(0., 360., n_wd, endpoint=False)
    site = IEA37Site(n_wt)
    wind_turbines = IEA37_WindTurbines()
    wake_model = IEA37SimpleBastankhahGaussian(site, wind_turbines)
    aep_calc = PyWakeAEP(wake_model)
    return aep_calc.get_TopFarm_cost_component(n_wt, wd=wd)


In [26]:
def optimize_iea37_locs(n_wt, n_wd, driver, state=None):
    """
    Parameters
    ----------
    - n_wt: int
        Number of wind turbines

    - n_wd: int
        Number of wind directions to consider for the AEP
        
    - driver: TopfarmDriver instance
        The optimization algorithm to use
        
    - state: dict(x=[], y=[]) [default=None]
        State to start from the optimization
    
    Returns
    -------
    - state: The state after the optimization

    """
    
    initial = get_iea37_initial(n_wt)
    design_vars = dict(zip('xy', (initial[:, :2]).T))
    
    tf = TopFarmProblem(
            design_vars,
            get_iea37_cost(n_wt, n_wd=n_wd),
            constraints=get_iea37_constraints(n_wt),
            driver=driver,
            plot_comp=XYPlotComp())
    
    if not state:
        _, state = tf.evaluate()
        
    _, state, _ = tf.optimize(state)
    return state, 

In [27]:
state = optimize_iea37_locs(9, n_wd=16, EasyScipyOptimizeDriver(disp=False))

The final optimized output is much lower than the value reported in Baker et al. (2019), which is 257.790 GWh. Moreover, the layout does not match the figures given in Appendix A in the same reference. This is due to the fact that the SLSQP optimizer was attracted to a local minimum. To find the global optimum, more advanced optimization procedures should be used. This benchmark is discussed in more detail in the validation report linked in TOPFARM's documentation.

In [15]:
state

{'x': array([-680.80198103, -854.32314245, -695.66777104,   89.8040452 ,
          11.54426432,  -64.22919445,  708.93660969,  860.87425685,
         677.89454027]),
 'y': array([-588.64986431,  283.07590553,  571.00468684, -895.50836595,
           9.60717992,  897.70519161, -554.44466226, -262.4795496 ,
         591.9957705 ])}